In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine, load_breast_cancer

wine1 = load_wine()
i = wine1.feature_names
wine_Y = wine1.target
wine = pd.DataFrame(data=wine1.data, columns=wine1.feature_names)

breast_cancer = load_breast_cancer()
b = breast_cancer.feature_names
breast_cancer_Y = breast_cancer.target
breast_cancer = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)

ionosphere=pd.read_csv('https://raw.githubusercontent.com/hargurjeet/MachineLearning/Ionosphere/ionosphere_data.csv')
ionosphere.rename(columns={'column_ai':'target'}, inplace=True)
# print(ionosphere)

wine_X = wine[i]
breast_cancer_X = breast_cancer[b]
ionosphere_X = ionosphere.drop(['target'], axis=1)
ionosphere_Y = ionosphere.target

In [23]:
import numpy as np
print(wine_X.min())
print(wine_X.head())
print(wine_X.dtypes)
print(np.issubdtype(wine_X.alcalinity_of_ash.dtype, np.integer))
def normalize_dataframe(df):
    # Subtract the minimum value from each column
    df_min = df.min()
    df_normalized = df - df_min

    # Divide by the range (max - min) of each column
    df_range = df.max() - df_min
    df_normalized /= df_range

    for column in wine1.feature_names:
        df_normalized[column] = df_normalized[column].astype('float32')

    return df_normalized

# Normalize the DataFrame
wine_X = normalize_dataframe(wine_X)
print(wine_X.head())
print(wine_Y)
print(wine_X.dtypes)

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int32
   alcohol  malic_acid  ash  alcalinity_of_ash  magnesium  total_phenols  \
0        0           0    0                  0          0              0   
1        0           0    0                  0          0              0   
2        0           0    0                  0          0              0   
3        0           0    0                  0          0              0   
4        0           0    0                  0          0              0   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity  hue  \
0         

In [24]:
from hmmlearn.hmm import MultinomialHMM
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from itertools import product
import numpy as np

state_range_values = [2, 3, 4, 5]
algorithm_values = ['viterbi', 'map']  # Choose the appropriate algorithm
random_state_values = [None]  # Set to None or an integer value for reproducibility
n_iter_values = [100, 150, 200, 300, 500]  # Adjust the number of iterations as needed
tol_values = [1e-3, 1e-4, 1e-5, 1e-6]  # Tolerance threshold for convergence
covariance_type_values = ['full', 'diag', 'spherical']  # Adjust covariance types
n_mix_values = [1, 2, 3]                  # Vary the number of Gaussian mixtures per state
init_params_values = ['s', 'm', 'sm', 'stmcw']      # Vary initialization parameters
min_covar_values = [1e-3, 1e-4, 1e-5]        # Vary the minimum covariance value
test_size_values = [0.1,0.2,0.3,0.4]


best_model = None
best_score = float('-inf')
best_train_test_ratio = 0.0
best_param_list = []
best_pred_y = 0
best_test_y = 0

for num_of_states,algo_type,random_state,n_iter,covariance_type,tol,test_size,n_mix,init_params,min_covar in product(state_range_values,algorithm_values, random_state_values, n_iter_values, covariance_type_values,tol_values,test_size_values, n_mix_values, init_params_values, min_covar_values):

    train_X, test_X, train_y, test_y = train_test_split(wine_X,wine_Y,random_state=None,test_size=test_size)
    params = set(np.array(train_X).ravel())
    model = MultinomialHMM(n_components=num_of_states,algorithm=algo_type,random_state=random_state,n_iter=n_iter, tol=tol, init_params=init_params,params=params)
    model.fit(train_X, train_y)
    pred_y = model.predict(test_y)
    score = accuracy_score(test_y,pred_y)

    if score > best_score:
        best_score = score
        best_train_test_ratio = test_size
        best_model = model
        best_param_list = [num_of_states,algo_type,random_state,n_iter,covariance_type,tol]
        best_pred_y = pred_y
        best_test_y = test_y

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


ValueError: Symbol counts should be nonnegative integers